# Adapter eval vs predict

### Introduction
While we train the models and evaluate them, all the processes are handled by those models and methods around them. 
The real behaviour of passing data to them (from outside) is not there and may differ during inference.

So it is crucial if the model will behave the same regardless of if evaluation and preprocessing is handled internally or externally.

### Methodology
We are testing if the models will produce similar MAP@50-95 results when evaluated using the `eval` method and the `predict` method.

### Results
We will run both `eval` and `predict` methods on the same dataset and compare the results. They should be very close to each other.

### Discussions
Values may differ slightly due to different thresholds used in `eval` and `predict`, but should be acceptably close (e.g., within 0.01-0.05 points - this is just an arbitrary observation).

In [1]:
from ml_carbucks import DATA_DIR
from ml_carbucks.adapters.EfficientDetAdapter import EfficientDetAdapter
from ml_carbucks.adapters.FasterRcnnAdapter import FasterRcnnAdapter
from ml_carbucks.adapters.UltralyticsAdapter import RtdetrUltralyticsAdapter, YoloUltralyticsAdapter
from ml_carbucks.ensemble.EnsembleModel import EnsembleModel
from ml_carbucks.utils.logger import setup_logger

logger = setup_logger("adapter_eval_vs_predict")

ensemble = EnsembleModel(
    classes=["scratch", "dent", "crack"],
    adapters=[
        YoloUltralyticsAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 32,
                "epochs": 27,
                "lr": 0.0015465639515144544,
                "momentum": 0.3628781599889685,
                "weight_decay": 0.0013127041660177367,
                "optimizer": "NAdam",
                "verbose": False,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_YoloUltralyticsAdaptermodel.pt",
        ),
        RtdetrUltralyticsAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 16,
                "epochs": 10,
                "lr": 0.0001141043015859849,
                "momentum": 0.424704619626319,
                "weight_decay": 0.00012292547851740234,
                "optimizer": "AdamW",
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_RtdetrUltralyticsAdaptermodel.pt",
        ),
        FasterRcnnAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 8,
                "epochs": 30,
                "lr_backbone": 2.6373762637681257e-05,
                "lr_head": 0.0011244046084737927,
                "weight_decay_backbone": 0.000796017512818448,
                "weight_decay_head": 0.0005747409908715994,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/FasterRcnnAdaptermodel.pth",
        ),
        EfficientDetAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 8,
                "epochs": 26,
                "optimizer": "momentum",
                "lr": 0.003459928723120903,
                "weight_decay": 0.0001302610542371722,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_EfficientDetAdaptermodel.pth",
        ),
    ],
).setup()

train_datasets = [
    (
        DATA_DIR / "car_dd_testing" / "images" / "train",
        DATA_DIR / "car_dd_testing" / "instances_train_curated.json",
    )
]
val_datasets = [
    (
        DATA_DIR / "car_dd_testing" / "images" / "val",
        DATA_DIR / "car_dd_testing" / "instances_val_curated.json",
    )
]

def test_eval_vs_predict(metric_name: str = "map_50_95"):
    eval_metrics = ensemble.evaluate_adapters_by_evaluation_from_dataset(val_datasets)  # type: ignore
    predict_metrics = ensemble.evaluate_adapters_by_predict_from_dataset(val_datasets)  # type: ignore

    for idx, adapter in enumerate(ensemble.adapters):
        logger.info(f"Adapter: {adapter.__class__.__name__}")
        logger.info(f"Metrics - evaluation: {round(eval_metrics[idx][metric_name], 4)}, predict: {round(predict_metrics[idx][metric_name], 4)}") # type: ignore

    return eval_metrics, predict_metrics



if __name__ == "__main__":
    m1, me2 = test_eval_vs_predict()

INFO ml_carbucks.adapters.UltralyticsAdapter 12:49:59 | Starting evaluation...
INFO ml_carbucks.adapters.UltralyticsAdapter 12:49:59 | Converting COCO annotations to YOLO format...
INFO ml_carbucks.adapters.UltralyticsAdapter 12:49:59 | COCO to YOLO conversion completed in 0.03 seconds
INFO ml_carbucks.adapters.UltralyticsAdapter 12:49:59 | YOLO dataset YAML created at: /home/bachelor/ml-carbucks/data/car_dd_testing/instances_val_curated.yaml
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 9181.5±1219.1 MB/s, size: 779.6 KB)
val: Scanning /home/bachelor/ml-carbucks/data/car_dd_testing/labels/val.cache... 633 images, 177 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 810/810 2.7Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 51/51 14.8it/s 3.4s0.1s
                   all        810       1406       0.43      0.399      0.375      0.199
Speed: 0.1ms preprocess, 3.1ms inference, 0.0ms loss, 0.3ms postprocess per image
R

100%|██████████| 102/102 [00:05<00:00, 19.09it/s]


INFO ml_carbucks.ensemble.EnsembleModel 12:50:56 | Evaluating adapter: RtdetrUltralyticsAdapter


100%|██████████| 102/102 [00:07<00:00, 13.00it/s]


INFO ml_carbucks.ensemble.EnsembleModel 12:51:04 | Evaluating adapter: FasterRcnnAdapter


100%|██████████| 102/102 [00:35<00:00,  2.91it/s]


INFO ml_carbucks.ensemble.EnsembleModel 12:51:40 | Evaluating adapter: EfficientDetAdapter


100%|██████████| 102/102 [00:13<00:00,  7.37it/s]


INFO adapter_eval_vs_predict 12:51:54 | Adapter: YoloUltralyticsAdapter
INFO adapter_eval_vs_predict 12:51:54 | Metrics - evaluation: 0.1994, predict: 0.1649
INFO adapter_eval_vs_predict 12:51:54 | Adapter: RtdetrUltralyticsAdapter
INFO adapter_eval_vs_predict 12:51:54 | Metrics - evaluation: 0.2758, predict: 0.2508
INFO adapter_eval_vs_predict 12:51:54 | Adapter: FasterRcnnAdapter
INFO adapter_eval_vs_predict 12:51:54 | Metrics - evaluation: 0.1037, predict: 0.0984
INFO adapter_eval_vs_predict 12:51:54 | Adapter: EfficientDetAdapter
INFO adapter_eval_vs_predict 12:51:54 | Metrics - evaluation: 0.1673, predict: 0.1661
